In [16]:
import weather_data as wd

import subprocess
import aiml
import os

In [17]:
# Create the kernel and learn AIML files
kernel = aiml.Kernel()
kernel.learn("start_bot.xml")
kernel.respond("load aiml")

kernel.setBotPredicate("name", "WeatherBot")

# Press CTRL-C to break this loop
while True:
    message = input("Enter your message >> ")
    if message == "quit":
        break #useful in notebook
        # exit() # useful in actual application
    elif message == "save":
        kernel.saveBrain("aiml/bot_brain.brn")
    else:
        bot_response = kernel.respond(message)

        if kernel.getPredicate("cur_weather_loc"):
            weather_dict = wd.weather_data(kernel.getPredicate("cur_weather_loc"))
            kernel.setPredicate("weather_descr", weather_dict["samenv"])
            kernel.setPredicate("temp", weather_dict["temp"])
            print(kernel.respond('CURRENT WEATHER LOC'))
        else:
            print(bot_response)

Loading start_bot.xml...done (0.00 seconds)
Loading aiml/basic_chat.aiml...done (0.00 seconds)
Loading aiml/weather_chat.aiml...done (0.00 seconds)
Enter your message >> hoe is het weer in Jaagpad 4 de kwakel
Momenteel is het geheel bewolkt en 10.1 °C in Jaagpad 4 de kwakel.
Enter your message >> quit


In [18]:
wd.weather_data("Jaagpad 4 de kwakel")['temp']

'10.1'